In [1]:
# Import useful packages
import os
import time
import pandas as pd
import glob
import numpy as np

# For loading/querying postgres
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import connect

In [2]:
# Create SQL engine to load data to postgres for central usage
from sqlalchemy import create_engine
dbname = '697_temp'
user = 'postgres'
password = 'poRter!5067'
host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com'

engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

### Loading Open Positions to Postgres

In [3]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Open_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Open_Positions_Data/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'Open_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'Open_Positions_Data/RF Reg_target_7_rebal_7_2017-01-01.csv',
 'Open_Positions_Data/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [4]:
# create temp_df to test loading; pick model not already in table
temp_df = pd.read_csv(txtfiles[2],index_col=0)
temp_df.head()

,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
0,Positions_2020-08-06,2020-08-06,PEAK,2256,59502.845678,26.375375,59502.845678,26.375375,0.0,RF Reg_target_7_rebal_7_2017-01-01
1,Positions_2020-08-06,2020-08-06,SPG,55,3221.555953,58.573745,3221.555953,58.573745,0.0,RF Reg_target_7_rebal_7_2017-01-01
2,Positions_2020-08-06,2020-08-06,KIM,273,2999.694877,10.987893,2999.694877,10.987893,0.0,RF Reg_target_7_rebal_7_2017-01-01
3,Positions_2020-08-06,2020-08-06,GILD,44,2907.976258,66.090369,2907.976258,66.090369,0.0,RF Reg_target_7_rebal_7_2017-01-01
4,Positions_2020-08-06,2020-08-06,KMI,215,2924.150363,13.600699,2924.150363,13.600699,0.0,RF Reg_target_7_rebal_7_2017-01-01


In [21]:
# Initial dataframe needs to create a new table
temp_df.to_sql('open_positions', engine)

In [5]:
# Append data to table
temp_df.to_sql('open_positions', engine, if_exists='append')

In [6]:
# Test with a load
def import_open_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM open_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['key', 'model'])
    return data
open_pos_df = import_open_positions().drop('index',axis=1)

open_pos_df.tail()


,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
15262,Positions_2021-07-23,2021-07-23,MSFT,1660,479446.871675,7800.583525,479556.175825,7801.753800,-0.022798,75percent_confidence_no_holding_15m_cnn
15263,Positions_2021-07-23,2021-07-23,NVDA,3503,685190.240400,5661.758775,684601.710775,5658.773425,0.085893,75percent_confidence_no_holding_15m_cnn
15264,Positions_2021-07-23,2021-07-23,QQQ,1472,539411.405900,9167.625925,539449.534375,9168.497675,-0.007069,75percent_confidence_no_holding_15m_cnn
15265,Positions_2021-07-23,2021-07-23,SPY,1155,506127.561600,10965.594350,506116.372000,10965.540200,0.002211,75percent_confidence_no_holding_15m_cnn
15266,Positions_2021-07-23,2021-07-23,TSLA,470,302600.480325,10940.374600,302589.959650,10942.778600,0.003477,75percent_confidence_no_holding_15m_cnn


### Loading track records to postgres

In [7]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("track_record/*.csv"):
    txtfiles.append(file)
    
txtfiles

['track_record/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'track_record/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'track_record/RF Reg_target_7_rebal_7_2017-01-01.csv',
 'track_record/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [8]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[2],index_col=0)
temp_df.head()

,Date,Value,Val_ex_cash,model
0,2020-08-06,100000.000000,99642.645275,RF Reg_target_7_rebal_7_2017-01-01
1,2020-08-07,101772.553411,101415.198685,RF Reg_target_7_rebal_7_2017-01-01
2,2020-08-10,104324.474630,103967.119905,RF Reg_target_7_rebal_7_2017-01-01
3,2020-08-11,105298.566802,104941.212077,RF Reg_target_7_rebal_7_2017-01-01
4,2020-08-12,104173.180481,103815.825756,RF Reg_target_7_rebal_7_2017-01-01


In [ ]:
# Initial dataframe needs to create a new table
temp_df.to_sql('track_record', engine)

In [9]:
# Append data to table
temp_df.to_sql('track_record', engine, if_exists='append')

In [10]:
# Test with a load
def import_track_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM track_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['Date', 'model'])
    return data
track_df = import_track_record().drop('index',axis=1)

track_df.head()


,Date,Value,Val_ex_cash,model
144,2020-07-19 00:00:00,100000.0000,100000.0000,75percent_confidence_no_holding_15m_cnn
145,2020-07-20 00:00:00,102126.2298,102126.2298,75percent_confidence_no_holding_15m_cnn
146,2020-07-21 00:00:00,101447.4265,101447.4265,75percent_confidence_no_holding_15m_cnn
147,2020-07-22 00:00:00,102702.7938,102702.7938,75percent_confidence_no_holding_15m_cnn
148,2020-07-23 00:00:00,102327.8937,102327.8937,75percent_confidence_no_holding_15m_cnn


### Loading cash records to postgres

In [11]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Cash_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Cash_Positions_Data/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'Cash_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'Cash_Positions_Data/RF Reg_target_7_rebal_7_2017-01-01.csv',
 'Cash_Positions_Data/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [12]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[2],index_col=0)
temp_df.tail()

,key,cash,model
149,cash_2021-03-11,5.490783,RF Reg_target_7_rebal_7_2017-01-01
150,cash_2021-03-12,5.490783,RF Reg_target_7_rebal_7_2017-01-01
151,cash_2021-03-15,5.490783,RF Reg_target_7_rebal_7_2017-01-01
152,cash_2021-03-16,5.490783,RF Reg_target_7_rebal_7_2017-01-01
153,cash_2021-03-17,5.490783,RF Reg_target_7_rebal_7_2017-01-01


In [13]:
# Initial dataframe needs to create a new table
temp_df.to_sql('cash_record', engine, if_exists='append')

In [14]:
# Test with a load
def import_cash_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM cash_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
cash_df = import_cash_record().drop('index',axis=1)

cash_df.tail()


,key,cash,model
418,cash_2021-03-11,5.490783,RF Reg_target_7_rebal_7_2017-01-01
419,cash_2021-03-12,5.490783,RF Reg_target_7_rebal_7_2017-01-01
420,cash_2021-03-15,5.490783,RF Reg_target_7_rebal_7_2017-01-01
421,cash_2021-03-16,5.490783,RF Reg_target_7_rebal_7_2017-01-01
422,cash_2021-03-17,5.490783,RF Reg_target_7_rebal_7_2017-01-01


In [15]:
cash_df.count()

key      566
cash     566
model    566
dtype: int64

### Loading Value Positions to postgres

In [16]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Val_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Val_Positions_Data/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'Val_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'Val_Positions_Data/RF Reg_target_7_rebal_7_2017-01-01.csv',
 'Val_Positions_Data/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [17]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[2],index_col=0)
temp_df.tail()

,key,value,model
149,val_2021-03-11,129090.678412,RF Reg_target_7_rebal_7_2017-01-01
150,val_2021-03-12,130598.742099,RF Reg_target_7_rebal_7_2017-01-01
151,val_2021-03-15,132633.335128,RF Reg_target_7_rebal_7_2017-01-01
152,val_2021-03-16,132407.606240,RF Reg_target_7_rebal_7_2017-01-01
153,val_2021-03-17,132356.878907,RF Reg_target_7_rebal_7_2017-01-01


In [18]:
# Initial dataframe needs to create a new table
temp_df.to_sql('val_positions', engine,if_exists='append')

In [19]:
# Test with a load
def import_val_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM val_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
val_df = import_val_positions().drop('index',axis=1)

val_df.tail()

,key,value,model
417,val_2021-03-11,129090.678412,RF Reg_target_7_rebal_7_2017-01-01
418,val_2021-03-12,130598.742099,RF Reg_target_7_rebal_7_2017-01-01
419,val_2021-03-15,132633.335128,RF Reg_target_7_rebal_7_2017-01-01
420,val_2021-03-16,132407.606240,RF Reg_target_7_rebal_7_2017-01-01
421,val_2021-03-17,132356.878907,RF Reg_target_7_rebal_7_2017-01-01
